In [1]:
import pyodbc
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MaxAbsScaler

import warnings
warnings.filterwarnings("ignore")

from pycaret.time_series import *
from datetime import datetime

In [2]:
def df_cluster(nits_clientes, fecha_final):
    # Conexion al dwh
    cnxn = pyodbc.connect(
        driver='{SQL Server}',
        server='192.168.100.58',
        uid='bilectura',
        pwd='D1sp@p3l3s')
    cursor = cnxn.cursor()

    df_SQL = []

    for nit in nits_clientes:
        #Consulta SQL
        consulta_SQL = f"SELECT DATEFROMPARTS(VTAANO, VTAMES, 1) AS 'Fecha', SUM(VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC WHERE CONCAT(CONCAT(VTANIT, '-'), VTASUC) = '{nit}' AND VTAFCH < '{fecha_final}' GROUP BY DATEFROMPARTS(VTAANO, VTAMES, 1)"

        #Carga de la data desde el dwh de Dispapeles y se guarda en df
        cursor.execute(consulta_SQL)
        rows = cursor.fetchall()
        df_SQL_int = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])
        df_SQL_int["Ventas"] = df_SQL_int["Ventas"].astype(int)
        df_SQL_int["Fecha"] = pd.to_datetime(df_SQL_int["Fecha"])

        df_SQL.append(df_SQL_int)

    #Convertir el arreglo, de 3D a 2D
    arr_df_SQL = np.array(df_SQL)
    n_filas = arr_df_SQL.shape[0] * arr_df_SQL.shape[1]
    df_SQL = arr_df_SQL.reshape(n_filas, 2)

    #Tranformaciones para convertir a pd DF
    df_SQL = pd.DataFrame(df_SQL, columns= ["Fecha", "Ventas"])
    df_SQL["Ventas"] = pd.to_numeric(df_SQL["Ventas"])
    # df_SQL["Fecha"] = pd.to_numeric(df_SQL["Fecha"])
    df_SQL = df_SQL.groupby("Fecha").sum()

    return df_SQL

In [3]:
df_clusters = pd.read_csv("C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Clustering/Clustering 27-1-23.csv",
                            encoding= 'utf-8', decimal= ",", sep= ";")
col_eliminar = ["Recency", "Monetary", "Frequency", "Distrito-Nombretipozona", "Cluster"]
df_clusters = df_clusters.drop(col_eliminar, axis= 1)

filtro_distrito = 10
filtro_tipozona = "Institucional"
filtro_cluster = "A"

df_clusters_f = df_clusters[
                    (df_clusters["Codigo distrito"] == filtro_distrito) &
                    (df_clusters["Nombre tipo zona"] == filtro_tipozona) &
                    (df_clusters["Letra cluster"] == filtro_cluster)
                ]

In [4]:
lista_nits = df_clusters_f["Nit cliente-sucursal"].tolist()
fecha_final = '2023-03-31'
fecha_final = datetime.strptime(fecha_final, '%Y-%m-%d').strftime('%Y-%m-%d')

In [5]:
ventas_cluster = df_cluster(nits_clientes= lista_nits, fecha_final= fecha_final)

In [6]:
ventas_cluster

,Ventas
Fecha,
2018-01-01,0
2018-02-01,57325910
2018-03-01,42296947
2018-04-01,61992674
2018-05-01,61996231
...,...
2022-11-01,485318911
2022-12-01,423495241
2023-01-01,516162793


In [7]:

s = setup(ventas_cluster, fh = 6, session_id = 123)

In [8]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:05:30
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [12]:
plot_model(best, plot = 'forecast')

In [10]:
plot_model(best, plot = 'diagnostics')

In [13]:
plot_model(best, plot = 'insample')

In [17]:
plot_model(best, plot = 'forecast', data_kwargs = {'fh': 9})

### Prediction

In [18]:
# predict on test set
holdout_pred = predict_model(best)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Seasonal Naive Forecaster,0.3773,0.2902,28042939.3333,42174310.2703,0.0625,0.0582,-1.1477


In [22]:
# generate forecast for 36 period in future
predict_model(best, fh = 9)

,y_pred
2022-10,513277088.0
2022-11,492987253.0
2022-12,517798934.0
2023-01,513277088.0
2023-02,492987253.0
2023-03,517798934.0
2023-04,513277088.0
2023-05,492987253.0
2023-06,517798934.0
